**Libraries**

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

<br>
<br>

## Reading the Data


### Database connection instance

In [ ]:
class SQLITE:

    def __init__(self, databaseuri: str):
        """
        
        :param databaseuri: The URI of a database file
        """
        
        self.databaseuri = databaseuri

    def connecting(self):
        """
        
        :return: database connection instance
        """

        connection = None

        try:
            connection = sqlite3.connect(self.databaseuri)
        except ConnectionError as err:
            raise Exception(err.strerror) in err

        return connection

In [ ]:
sqlite = SQLITE(databaseuri='../data/gpinhours.sqlite')

<br>
<br>

### Data reading

In [ ]:
connection = sqlite.connecting()
query = 'SELECT * FROM inhours'
inhours = pd.read_sql_query(query, connection)
connection.close()

In [ ]:
inhours.info()

<br>
<br>

## Data Cleaning

In [ ]:
inhours.head()

<br>
<br>

### Fix the population count

<br>

**Create function**

In [ ]:
def as_number_or_nan(x):
    return pd.to_numeric(x, errors='coerce')
    

In [ ]:
inhours.loc[:, 'POP'] = as_number_or_nan(x = inhours.TOTAL_POP)

In [ ]:
inhours.head()

<br>

**Missing values**

In [ ]:
inhours.loc[np.isnan(inhours.POP), 'NAME'].value_counts()

<br>
<br>

### Fix the date

In [ ]:
inhours.loc[:, 'date'] = pd.to_datetime(inhours.lastdate, unit='D', origin='unix')

In [ ]:
inhours.date.describe(datetime_is_numeric=True)

<br>
<br>

### Trim the years

In [ ]:
inhours.loc[:, 'year'] = inhours.date.dt.year

In [ ]:
inhours.year.value_counts()

In [ ]:
inhours.shape

<br>

Reduce

In [ ]:
inhours = inhours.loc[inhours.year != 2020, :]

In [ ]:
inhours.year.value_counts()

In [ ]:
inhours.shape

<br>
<br>

### Drop zero populations

In [ ]:
inhours = inhours.loc[inhours.POP != 0, :]

In [ ]:
inhours = inhours.loc[~np.isnan(inhours.POP), :]

<br>
<br>

### Final clean data

In [ ]:
inhours

<br>
<br>

## North South Divide

<br>

### Read the table

In [ ]:
connection = sqlite.connecting()
query = 'SELECT * FROM localauth'
localauth = pd.read_sql_query(query, connection)
connection.close()

In [ ]:
localauth.info()

In [ ]:
localauth.head()

<br>

### Merge with GP data

In [ ]:
inhoursNS = inhours.merge(localauth, how='left', on='NAME')

In [ ]:
inhoursNS

<br>
<br>

### Group and aggregate

In [ ]:
fields = ['NS', 'year', 'POP', 'Flu_OBS', 'Vom_OBS', 'Diarr_OBS', 'Gastro_OBS']

In [ ]:
# Either
# inhoursNS[fields].groupby(by=['NS', 'year']).agg('sum')


# Or
aggNS = inhoursNS[fields].groupby(by=['NS', 'year']).agg(Total=('POP', sum), 
                                                         Flu_OBS=('Flu_OBS', sum), 
                                                         Vom_OBS=('Vom_OBS', sum), 
                                                         Diarr_OBS=('Diarr_OBS', sum), 
                                                         Gastro_OBS=('Gastro_OBS', sum))
aggNS

<br>
<br>

### Consultation rates